# Phishing with Amazon SES

This iPython notebook was created to make phishing campaigns a little less painful. We use AWS, in particular Simple Email Service to send our phishing campaign. 

Stuff you need to take care of OR warning: this script doesn't do it all:

1. Go register a domain and set up email like the pro you are. Make sure it all works. 
1. You have set up your own phishing web site complete with appropriate logging so you can capture visits. Any and all features you want (eg. save submitted usernames and passwords) must be implemented by YOU. See [notes](#notes) for help!
1. You have crafted [an HTML email](#email) that works properly when rendered (likely in Outlook... which can be a pain). Hint: use the test feature of this script and tweak the email accordingly. You can craft and test the email in this notebook now too, or build it externally using some other editor and then just load it. 
1. You have the Unix CLI fu (or Python fu) to assemble statistics on clicks yourself.

In [144]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from itertools import cycle, izip
from IPython.display import Image, HTML, display
from ipywidgets import widgets, interact, interactive, fixed
from bs4 import BeautifulSoup
import markdown2
import pandas as pd
import boto.ses
import os

## Amazon Setup.
You'll need to get these from Amazon Web Services (AWS): 

In [145]:
aws_zone='us-east-1'
aws_id='YOUR_ID_HERE'
aws_secret='YOUR_SECRET_HERE'

# Create the SES connection object
conn = boto.ses.connect_to_region(aws_zone, aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)

## Phish Configuration
Provide the following configuration information for the phishing campaign:

- **email_list**: The location of the Excel file containing phishing target information. Format is first, last, email
- **msg_subject**: The subject line of the phishing email.
- **msg_from**: The from address format is: `Name <email@domain.com>`
- **phish_url**: The base URL for phishing (a unique ID will based on email address will be appended.
- **test_email**: The email address to use for sending tests of the phishing campaign.
- **html_file**: The location of a file containing the html formatted email, use {{phish}} to insert phishing URL and {{firstName}} to insert the target's firstname.
- **img_file**: The location of image file(s) to be read and added as a MIME attachments. 

In [146]:
# example DOS style path: C:\\Users\\username\\Documents\\Test\\targets.xlsx

email_list = 'C:\\Users\\blark\\Documents\\Test\\dummy.xlsx'
msg_subject = 'This is the message subject'
msg_from = 'derp@herp.com'
phish_url = 'https://www.secure-company.com'
test_email = 'your@email.com'
img_files = [] # don't forget to use two \\ for DOS style paths.

### Read image files and convert them to MIME attachments.

This loop will read the images, and convert them to MIME and then display them with a label for your reference. 

Insert these files into your HTML template using an image tag like so: `<img src="cid:image1">`


In [147]:
msg_img = []

for idx, f in enumerate(img_files):
    img = open(f, 'rb').read()
    msg_img.insert(idx, MIMEImage(img, name=os.path.basename(f)))
    msg_img[idx].add_header('Content-ID', '<image%s>' % idx)
    display('<Image' + str(idx) + '>', Image(img))

## Email Body

<a id="email"></a>

If you want to edit the body in browser you can simply enter it here. This field is prepopulated with Lorum Ipsum for demonstration purposes. 

Thanks to the markdown2 module you can use [markdown syntax](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) to make things more readable, and when that's not powerful enough you can switch to inline HTML.

In [148]:
# This will be added to all <p> elements for your convenience.
# Unless there is already a style tag, then you're on your own. 
BODY_STYLE = "font-family:sans-serif;"

# **********************************************************************************

email_body = """
Dear {{firstName}},

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua [Link Test]({{phish}}).

Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Praesent non turpis lacus. Aenean hendrerit dui justo, a euismod mi luctus ac. Morbi sed tortor vel ante tempor pulvinar. Ut mattis est nisl, ac placerat arcu tincidunt non. Donec condimentum molestie tempus. Suspendisse condimentum feugiat leo dapibus viverra. Sed nec porttitor augue, nec varius augue. Phasellus enim justo, molestie vehicula dictum vel, convallis id diam. 
"""

# **********************************************************************************
#
# Convert markdown to HTML, add some style with BeautifulSoup
#
html_body = markdown2.markdown(email_body)

html_body = BeautifulSoup(html_body) 
# strip all the tags and store text only for later
text_body = html_body.getText()
# add some style to the paragraph elements
for p_tag in html_body.find_all('p'):
    if p_tag.has_attr('style'):
        pass
    else:
        p_tag['style'] = BODY_STYLE
        
html_body = html_body.prettify("utf-8")

### Email Signature

This is where things get dicey. Companies like their fancy signatures that have all kinds of images and shit, they can be hard to emulate. Godspeed. 

Usually this is where you want to insert your images, do so using markdown like so: `![Alt Text](cid:image0)`

**Note:** if you want to avoid designing the email in the iPython Notebook you can do it all outside and just read the file in using the last line of the following cell that is commented out. Be sure to delete all this other cruft if you do that. 

In [149]:
SIG_STYLE = 'font-family:sans-serif;'

# **********************************************************************************

email_sig = """
--  
**IT Helpdesk**  
[helpdesk@secure-company.com](mailto:helpdesk@secure-company.com)  

T: (888)PHI-SHME  
A: 123 Derp St., Suite 1000 | Anytown, ON | H0H 0H0  
[http://www.company.com](http://www.company.com)  
![Image Examle](cid:image0)
___
<table style="margin-top: 1em;border-collapse:collapse;"><tbody>
<tr>
<td>
Inline HTML table
</td>
<td style="padding-left:10px"><p>Some stupid disclaimer notice here.</p>
</tr>
</tbody></table>
"""

# **********************************************************************************
#
# Convert markdown to HTML and add style 
#
html_sig = markdown2.markdown(email_sig)

html_sig = BeautifulSoup(html_sig)
# strip all the tags and store text only for later
text_sig = html_sig.getText()

# you can do other customizations here if you need to, example:
# this is pretty derpy, I should probably build a markdown extension for this...
#for img_tag in html_sig.find_all('img'):
#    img_tag['style'] = 'float:left'
for p_tag in html_sig.find_all('p'):
    if p_tag.has_attr('style'):
        pass
    else:
        p_tag['style'] = SIG_STYLE
html_sig = html_sig.prettify("utf-8")

# Finally, assemble the email - text only version is unused at this time.
text_email = text_body + text_sig 
html_email = html_body + html_sig

# alternately you could just open and read the email html file
#html_email = open(HTMLfile, 'r').read()

### Lets see what the email looks like...

**Note:** The image won't show in Notebook, but it will when you send the email!

In [150]:
display(HTML(html_email))

Inline HTML table,Some stupid disclaimer notice here.


## Run all

### At this time it should be safe to run all cells...don't worry anything that sends email requires manual user interaction. 

## Helper Functions

- **cypher** encodes/decodes addresses into an ID, the cypher part helps disguise the address so someone smart enough to try and decode base64 will get garbage. Set encode to False to decode a string, otherwise just provide a string and this function will encode it. We're not trying to achieve real security with this, just make it a bit tougher to decode for a regular user.

- **phish** assembles and sends the email through SES - you could really easily change the last few lines of this if you don't want to use AWS, there are a million examples on how to send email in Python ... Google is your friend. 

In [162]:
def cypher(text, encode=True):
    key = 'qUpUph6vut=e'
    if not encode: text = text.decode('base64')
    cyphered = ''.join(chr(ord(c)^ord(k)) for c,k in izip(text, cycle(key)))
    if encode: cyphered = cyphered.encode('base64', 'strict')
    return cyphered

def phish(to_addr=test_email, first_name="Test", sender=msg_from, url=phish_url, text=text_email, html=html_email):
    global df
    # assemble the phishing link, adding an id parameter for identification of clicks
    uid = cypher(to_addr)
    rl = url + '?id=' + uid
    
    # replace appropriate text, verbose code for clarity.
    text = text.replace('{{firstName}}', first_name).replace('{{phish}}', phish_url)
    html = html.replace('{{firstName}}', first_name).replace('{{phish}}', phish_url)
    
    # Create message container.
    msg = MIMEMultipart('related')
    msg['Subject'] = msg_subject
    msg['From'] = msg_from
    msg['To'] = to_addr

    # Attach message parts
    msg.attach(MIMEText(html, "html"))    
    # Attach image(s)
    for idx, inline_image in enumerate(msg_img):   
        msg.attach(inline_image)

    # Send the email through AWS SES.
    try:
        pass
        # ************************** uncomment to actually send mail *************************
        #conn.send_raw_email(source=sender, raw_message=msg.as_string(), destinations=to_addr)
        # ************************************************************************************
    except Exception as e:
        success = "False"
        print("Couldn't send message! Error: %s %s" % e.message, e.args)
    else:
        success = "True"
    finally:
        return success, uid

## Test, test, and test again! 

Click the button below to send a test email. 

In [152]:
def testEmail(btn):
    print('Sending test email...')
    phish(sender=msg_from, first_name="Testuser", to_addr=test_email, url=phish_url, html=html_email)

btn = widgets.Button(description="Click here to send a test email...", icon="check")
btn.on_click(testEmail)
display(btn)

## Read in the target file (XLS)

We read in a list of targets from an XLS file using Pandas. Then display a little bit of information about the list that we just read...

In [159]:
# Some simple logic to make sure the list has been specified
if email_list:
    df = pd.read_excel(io=email_list, sheetname=0, parse_cols=[0,1,2])
    print("%d email addresses loaded, sample data: " % len(df.index))
    # add some columns for tracking
    df['sent'] = ""
    df['uid'] = ""
    display(df.head())
    # uncomment the following two lines to see all emails
    #for idx, (last, first, email) in df.iterrows():
    #    print first, last, cypher(email)

1 email addresses loaded, sample data: 


,first,last,email,sent,uid
0,test,user,test@email.com,,


In [163]:
sent = False

def goPhishing(wut):
    global sent
    if (armed.value and sent is False):
        print "Casting rod..."
        sent = True
        for idx, (first, last, email, _, _) in df.iterrows():
            # Remove any spaces and incorrectly formatted data
            first = first.capitalize().strip()
            last = last.capitalize().strip()
            email = email.strip()
            print("Sending phish to %s %s (%s)." %(first, last, email)),
            # I am paranoid so this is TRIPLE saftey.. uncomment the next line to actually send.
            success, uid = phish(sender=msg_from, first_name=first, to_addr=email, url=phish_url, html=html_email)
            if success:
                print("Sent: %s" % uid.rstrip())
                df["sent"][idx] = success
                df["uid"][idx] = uid
            
    elif (armed.value and sent):
        print "Gear down big rig, emails have already been sent!"
    else:
        print "Refusing to send, you must turn of safety first..."
    
armed = widgets.ToggleButton(description="Click here to turn off safety!", tooltip="Safey switch - emails won't send unless you click this first.")
display(armed)

go_button = widgets.Button(description="SEND EMAILS", tooltip="WARNING - MAKE SURE YOU REALLY WANT TO DO THIS!")
go_button.on_click(goPhishing)
display(go_button)

Casting rod...
Sending phish to Test User (test@email.com). Sent: BTADITANWxccGBMGHjg=


## Save Excel file with UID and confirmation

TODO: Read in unique hits and password files to add data. 

In [164]:
# Run this cell after emails are sent to see updated datatable...
if sent:
    display(df.head())
    writer = pd.ExcelWriter(os.path.dirname(email_list) + '\\' + 
                            os.path.basename(email_list).split(".")[0] + 
                            '_done.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()

,first,last,email,sent,uid
0,test,user,test@email.com,True,BTADITANWxccGBMGHjg=\n


## Decode example
You can easily decode the ID from your logs if you want to see who is clicking on the link.

In [156]:
print cypher("test@email.com")

BTADITANWxccGBMGHjg=



In [157]:
print cypher("BTADITANWxccGBMGHjg=\n", False)

test@email.com


# Notes and Resources

<a id="notes"></a>

1. Example Apache SSL configuration (replace SITENAME with client name and DOMAIN_NAME with FQDN)

  ```
    <VirtualHost *:443>
      ServerName www.SITENAME.com
      ServerAlias SITENAME.com
      DocumentRoot /var/www/SITENAME/

      CustomLog /var/log/apache2/SITENAME.log combined
      ErrorLog /var/log/apache2/SITENAME.log

      SSLEngine on
      # Refer to https://cipherli.st/ for SSL/TLS security best practice. 
      SSLCipherSuite EECDH+AESGCM:EDH+AESGCM:AES256+EECDH:AES256+EDH
      SSLProtocol All -SSLv2 -SSLv3
      SSLHonorCipherOrder On
      # Turn on mod-headers
      Header always set Strict-Transport-Security "max-age=63072000; includeSubdomains; preload"
      Header always set X-Frame-Options DENY
      Header always set X-Content-Type-Options nosniff
      # Requires Apache >= 2.4
      #SSLSessionTickets Off
      #SSLCompression off
      #SSLUseStapling on
      #SSLStaplingCache "shmcb:logs/stapling-cache(150000)"

      SSLCertificateFile "/etc/letsencrypt/live/DOMAIN_NAME/cert.pem"
      SSLCertificateKeyFile "/etc/letsencrypt/live/DOMAIN_NAME/privkey.pem"
      SSLCertificateChainFile "/etc/letsencrypt/live/DOMAIN_NAME/fullchain.pem"
    </VirtualHost>
    ```

2. Let's Encrypt is an easy and free way to get a valid certificate

  ```
sudo -s
service apache2 stop
git clone https://github.com/letsencrypt/letsencrypt
cd letsencrypt
./letsencrypt-auto certonly --standalone -d example.com -d www.example.com
service apache2 start
```

3. Site mirroring 

  ```
wget --no-check-certificate -nd -k -p -erobots=off https://SITENAME/
```

4. Capturing creds with PHP and redirect back to main page with error message (templates):

  * [Error page](https://gist.github.com/blark/4c211940be01582a2729)
  * [Capture form data](https://gist.github.com/blark/1ff83db00b7e49cb17ed)
